# 종속성 설치

In [ ]:
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade--quiet tokenizers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgrade--quiet


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 33.5 MB/s eta 0:00:00


# LLM 모델 설치

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 ID 설정
base_model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

# 2. BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, # 4bit quantization으로 모델을 불러와 메모리 소모를 줄임
)

# 3. 모델 로드
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
# 4. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,  # 최대 토큰 길이
    padding_side="left",   # 입력 패딩 방향
    add_eos_token=True     # EOS 토큰 추가
)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# LLM 파이프라인 설정 (LangChain과 통합)
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-9-0e279de07470>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


# 질의 증강 프롬프트 엔지니어링

In [ ]:
PROMPT_QUERY_AUGMENTED = """
당신은 언어공학 및 대화분석 전문가로서, 사용자의 질의 의도를 분석하고, 기존 질의에 사용자 의도가 더 잘 반영되도록 질의를 업그레이드하는 역할을 맡고 있습니다.
아래의 <지침>을 따르고, <예시>를 참고하세요.

<지침>
1. 사용자 질의에서 핵심 키워드를 추출합니다.
2. 사용자의 의도를 분석합니다. (사용자가 무엇을 알고 싶어하는지 생각해보세요.)
3. 추출된 키워드와 분석된 의도를 바탕으로, 구체적이고 명확한 질의를 1개만 업그레이드 합니다.
4. 주어진 사용자 질의에 대해서만 답변하고 추가적인 질의를 생성하지 않습니다.
</지침>

<예시1>
1. 사용자 질의: "거시경제학과 케인즈",
2. 업그레이드 질의: "케인즈의 거시 경제학 이론이 현대 경제 정책에 어떤 영향을 미쳤는지 설명해주세요."
</예시1>

<예시2>
1. 사용자 질의: "미시경제학과 정부개입,
2. 업그레이드 질의: "미시경제학에서 시장 실패를 해결하기 위한 정부 개입의 주요 사례는 무엇인가요?"
</예시2>


<지정 응답 형식>:
1. 사용자 질의: {question},
2. 업그레이드 질의:[이 자리에는 실제 생성한 업그레이드 질의가 들어가야 합니다.];
========================= 실제 생성 종료 =======================================
</지정 응답 형식>

반드시 위 <지정 응답 형식>만 사용하여 작성하세요. 형식을 벗어나거나 불필요한 추가 정보를 포함하지 마세요.
지정 응답 형식에 따른 생성이 끝나면 추가, 반복 생성 없이 종료하세요.
========================= 실제 생성 ============================================
"""

In [ ]:
from langchain_core.prompts import PromptTemplate
# PromptTemplate 설정
prompt = PromptTemplate(
    template=PROMPT_QUERY_AUGMENTED,
    input_variables=["question"]
)

In [ ]:
# 사용자 질의 테스트
query = "LLM과 SOTA"
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


당신은 언어공학 및 대화분석 전문가로서, 사용자의 질의 의도를 분석하고, 기존 질의에 사용자 의도가 더 잘 반영되도록 질의를 업그레이드하는 역할을 맡고 있습니다.
아래의 <지침>을 따르고, <예시>를 참고하세요.

<지침>
1. 사용자 질의에서 핵심 키워드를 추출합니다.
2. 사용자의 의도를 분석합니다. (사용자가 무엇을 알고 싶어하는지 생각해보세요.)
3. 추출된 키워드와 분석된 의도를 바탕으로, 구체적이고 명확한 질의를 1개만 업그레이드 합니다.
4. 주어진 사용자 질의에 대해서만 답변하고 추가적인 질의를 생성하지 않습니다.
</지침>

<예시1>
1. 사용자 질의: "거시경제학과 케인즈" 
2. 업그레이드 질의: "케인즈의 거시 경제학 이론이 현대 경제 정책에 어떤 영향을 미쳤는지 설명해주세요."
</예시1>

<예시2>
1. 사용자 질의: "LLM 모델 특징" 
2. 업그레이드 질의: "LLM 모델들의 주요 특징(모델 크기, 학습 데이터, 구조 등)과 각각의 장단점을 비교 분석하여 설명해주세요."
</예시2>

<지정 응답 형식>:
1. 사용자 질의: LLM과 SOTA
2. 업그레이드 질의:[기존 질의에 사용자 의도가 더 잘 반영되도록 질의를 업그레이드]
========================= 실제 생성 종료 =======================================
</지정 응답 형식>

반드시 위 <지정 응답 형식>만 사용하여 작성하세요. 형식을 벗어나거나 불필요한 추가 정보를 포함하지 마세요.
지정 응답 형식에 따른 생성이 끝나면 추가, 반복 생성 없이 종료하세요.
========================= 실제 생성 ============================================
1. 사용자 질의: LLM과 SOTA
2. 업그레이드 질의: LLM과 SOTA 모델들 간의 주요 특징(모델 크기, 학습 데이터, 구조 등)과 각각의 장단점을 비교 분석하여 설명해주세요.
========================

In [ ]:
# 사용자 질의 테스트
query = "시간복잡도와 프로그램"
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


당신은 언어공학 및 대화분석 전문가로서, 사용자의 질의 의도를 분석하고, 기존 질의에 사용자 의도가 더 잘 반영되도록 질의를 업그레이드하는 역할을 맡고 있습니다.
아래의 <지침>을 따르고, <예시>를 참고하세요.

<지침>
1. 사용자 질의에서 핵심 키워드를 추출합니다.
2. 사용자의 의도를 분석합니다. (사용자가 무엇을 알고 싶어하는지 생각해보세요.)
3. 추출된 키워드와 분석된 의도를 바탕으로, 구체적이고 명확한 질의를 1개만 업그레이드 합니다.
4. 주어진 사용자 질의에 대해서만 답변하고 추가적인 질의를 생성하지 않습니다.
</지침>

<예시1>
1. 사용자 질의: "거시경제학과 케인즈",
2. 업그레이드 질의: "케인즈의 거시 경제학 이론이 현대 경제 정책에 어떤 영향을 미쳤는지 설명해주세요."
</예시1>

<예시2>
1. 사용자 질의: "미시경제학과 정부개입, 
2. 업그레이드 질의: "미시경제학에서 시장 실패를 해결하기 위한 정부 개입의 주요 사례는 무엇인가요?"
</예시2>


<지정 응답 형식>:
1. 사용자 질의: 시간복잡도와 프로그램,
2. 업그레이드 질의:[이 자리에는 실제 생성한 업그레이드 질의가 들어가야 합니다.];
========================= 실제 생성 종료 =======================================
</지정 응답 형식>

반드시 위 <지정 응답 형식>만 사용하여 작성하세요. 형식을 벗어나거나 불필요한 추가 정보를 포함하지 마세요.
지정 응답 형식에 따른 생성이 끝나면 추가, 반복 생성 없이 종료하세요.
========================= 실제 생성 ============================================
1. 사용자 질의: 시간복잡도와 프로그램,
2. 업그레이드 질의: 시간 복잡도와 관련된 알고리즘 프로그램의 예시와 설명이 필요합니다.
========================= 실제 생성 종료 ==============================

In [ ]:
# 사용자 질의 테스트
query = "조선시대 백자와 청자"
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


당신은 언어공학 및 대화분석 전문가로서, 사용자의 질의 의도를 분석하고, 기존 질의에 사용자 의도가 더 잘 반영되도록 질의를 업그레이드하는 역할을 맡고 있습니다.
아래의 <지침>을 따르고, <예시>를 참고하세요.

<지침>
1. 사용자 질의에서 핵심 키워드를 추출합니다.
2. 사용자의 의도를 분석합니다. (사용자가 무엇을 알고 싶어하는지 생각해보세요.)
3. 추출된 키워드와 분석된 의도를 바탕으로, 구체적이고 명확한 질의를 1개만 업그레이드 합니다.
4. 주어진 사용자 질의에 대해서만 답변하고 추가적인 질의를 생성하지 않습니다.
</지침>

<예시1>
1. 사용자 질의: "거시경제학과 케인즈",
2. 업그레이드 질의: "케인즈의 거시 경제학 이론이 현대 경제 정책에 어떤 영향을 미쳤는지 설명해주세요."
</예시1>

<예시2>
1. 사용자 질의: "미시경제학과 정부개입, 
2. 업그레이드 질의: "미시경제학에서 시장 실패를 해결하기 위한 정부 개입의 주요 사례는 무엇인가요?"
</예시2>


<지정 응답 형식>:
1. 사용자 질의: 조선시대 백자와 청자,
2. 업그레이드 질의:[이 자리에는 실제 생성한 업그레이드 질의가 들어가야 합니다.];
========================= 실제 생성 종료 =======================================
</지정 응답 형식>

반드시 위 <지정 응답 형식>만 사용하여 작성하세요. 형식을 벗어나거나 불필요한 추가 정보를 포함하지 마세요.
지정 응답 형식에 따른 생성이 끝나면 추가, 반복 생성 없이 종료하세요.
========================= 실제 생성 ============================================
1. 사용자 질의: 조선시대 백자와 청자,
2. 업그레이드 질의: 조선시대 백자와 청자의 제작 기술과 특징은 무엇인가요?
========================= 실제 생성 종료 =====================================